In [2]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import BatchNormalization, Input,Conv2D, Activation, MaxPooling2D,MaxPooling2D,AveragePooling2D, Dense, GlobalAveragePooling2D
from keras import optimizers
from keras.layers import Dropout, Flatten, Concatenate
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
import keras_resnet.models
import keras
from keras import callbacks
from keras.models import model_from_json
import h5py
tbCallBack = callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)

Using TensorFlow backend.


In [4]:
df = pd.read_csv('legend.csv',header = None)

print(df.head())

         0                                1         2
0  user.id                            image   emotion
1      628  facial-expressions_2868588k.jpg     anger
2      628  facial-expressions_2868585k.jpg  surprise
3      628  facial-expressions_2868584k.jpg   disgust
4      628  facial-expressions_2868582k.jpg      fear


In [5]:
X = np.stack([cv2.imread('finalimages\\'+str(img)) for img in df.iloc[10:4010,1]]).astype(np.float)[:,:,:,np.newaxis]
x_t = np.stack([cv2.imread('finalimages\\'+str(img1)) for img1 in df.iloc[4100:4600,1]]).astype(np.float)[:,:,:,np.newaxis]

In [5]:
y = np.vstack(df.iloc[10:4010,-1].values)
y_t = np.vstack(df.iloc[4100:4600,-1].values)

In [6]:
y_train = y.reshape(4000,)
y_test = y_t.reshape(500,)

In [14]:
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# convert integers to dummy variables (i.e. one hot encoded)
y1 = np_utils.to_categorical(encoded_Y)
encoder.fit(y_train)
encoded_Y1 = encoder.transform(y_test)
# convert integers to dummy variables (i.e. one hot encoded)
y2 = np_utils.to_categorical(encoded_Y1)
print(encoded_Y1)

[5 5 5 5 4 0 4 5 5 5 5 5 7 4 5 5 7 5 4 4 4 4 5 5 5 5 5 5 5 5 4 4 5 5 4 5 5
 5 5 5 5 5 7 5 5 5 5 5 7 5 7 5 5 0 5 7 5 7 7 5 5 5 5 5 5 5 5 5 5 5 5 5 4 5
 5 5 5 5 5 5 5 5 5 5 4 5 5 4 5 5 5 5 4 5 4 5 5 5 5 5 5 5 5 5 4 7 5 5 5 5 4
 5 5 5 5 5 4 4 5 5 5 4 4 5 4 5 5 5 5 5 5 6 5 5 5 5 4 4 5 4 5 5 5 5 5 4 5 5
 5 5 5 5 5 5 5 5 5 5 4 5 5 5 5 5 5 7 5 5 5 5 5 5 4 5 5 5 4 5 5 4 4 5 5 7 5
 7 5 4 5 0 4 4 4 5 5 5 5 4 5 5 0 5 5 4 7 5 4 0 7 5 5 7 5 5 5 5 5 7 5 5 5 5
 5 4 5 4 4 5 6 4 5 5 5 5 4 4 7 4 4 4 7 5 5 5 5 4 5 5 5 5 4 4 4 4 4 6 4 4 5
 4 6 5 4 4 5 5 4 4 4 4 5 4 5 4 5 5 5 4 4 4 5 5 4 4 5 5 5 5 5 6 4 5 5 4 4 5
 5 5 5 5 4 4 5 4 5 5 5 4 4 4 4 5 5 4 5 5 0 5 4 4 4 4 5 5 5 4 4 4 4 4 4 4 4
 5 5 5 5 0 4 5 0 5 5 4 5 4 5 5 5 5 4 4 5 5 7 4 4 5 4 4 4 4 5 5 5 5 5 5 5 5
 3 5 5 5 5 4 4 4 4 5 4 4 5 5 4 5 5 5 5 4 4 4 4 4 5 5 4 4 4 4 5 4 5 4 5 4 4
 4 5 4 0 4 5 5 4 5 5 4 5 0 5 4 4 5 5 7 5 4 4 4 4 5 5 4 7 4 4 4 4 4 5 4 4 4
 4 4 4 4 5 5 5 5 5 4 7 4 4 5 5 4 5 4 4 4 4 4 4 5 4 5 4 5 5 4 5 5 5 5 5 5 5
 4 4 5 4 4 5 5 4 5 5 4 4 

In [7]:
X_train = X / 255
X_test = x_t / 255

In [8]:
X_train = X_train.reshape(4000,128,128,3)
X_test = X_test.reshape(500,128,128,3)

In [9]:
inputs = Input(shape=(128,128,3))
num_classes = 8
num_filter = 64
x1 = Conv2D(8,(7,7),strides=(2,2),kernel_initializer='glorot_uniform',activation='relu')(inputs)
x1 = MaxPooling2D(pool_size=(3,3),padding='same', strides = (2,2))(x1)
x1 = BatchNormalization()(x1)
x1 = Conv2D(16,(1,1),strides=(1,1),kernel_initializer='glorot_uniform',activation='relu')(x1)
x1 = Conv2D(32,(3,3),strides=(1,1),kernel_initializer='glorot_uniform',activation='relu')(x1)
x1 = BatchNormalization()(x1)
x1 = MaxPooling2D(pool_size=(3,3),padding='same', strides = (2,2))(x1)

tower1 = Conv2D(filters = num_filter, kernel_size=(1,1),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x1)
tower1 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(tower1)

tower2 = Conv2D(filters = num_filter, kernel_size=(1,1),padding='same',kernel_initializer='glorot_uniform',activation='relu')(x1)
tower2 = Conv2D(filters = num_filter, kernel_size=(5,5),padding='same',kernel_initializer='glorot_uniform', activation='relu')(tower2)

tower3 = MaxPooling2D(pool_size=(3,3),padding='same', strides = (1,1))(x1)
tower3 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(tower3)
tower4 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x1)
x2 = keras.layers.concatenate([tower1,tower2,tower3,tower4],axis=1)
x2 = MaxPooling2D(pool_size=(3,3),padding='same', strides = (2,2))(x2)

tower4 = Conv2D(filters = num_filter, kernel_size=(1,1),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x2)
tower4 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(tower4)

tower5 = Conv2D(filters = num_filter, kernel_size=(1,1),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x2)
tower5 = Conv2D(filters = num_filter, kernel_size=(5,5),padding='same',kernel_initializer='glorot_uniform',activation='relu')(tower5)

tower6 = MaxPooling2D(pool_size=(3,3),padding='same', strides = (1,1))(x2)
tower6 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform',activation='relu')(tower6)

tower7 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x2)
x3 = keras.layers.concatenate([tower4,tower5,tower6,tower7],axis=1)

tower8 = Conv2D(filters = num_filter, kernel_size=(1,1),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x3)
tower8 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(tower8)

tower9 = Conv2D(filters = num_filter, kernel_size=(1,1),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x3)
tower9 = Conv2D(filters = num_filter, kernel_size=(5,5),padding='same',kernel_initializer='glorot_uniform',activation='relu')(tower9)

tower10 = MaxPooling2D(pool_size=(3,3),padding='same', strides = (1,1))(x3)
tower10 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform',activation='relu')(tower10)

tower11 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x3)
x4 = keras.layers.concatenate([tower8,tower9,tower10,tower11],axis=1)
tower12 = Conv2D(filters = num_filter, kernel_size=(1,1),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x4)
tower12 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(tower12)

tower13 = Conv2D(filters = num_filter, kernel_size=(1,1),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x4)
tower13 = Conv2D(filters = num_filter, kernel_size=(5,5),padding='same',kernel_initializer='glorot_uniform',activation='relu')(tower13)

tower14 = MaxPooling2D(pool_size=(3,3),padding='same', strides = (1,1))(x4)
tower14 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform',activation='relu')(tower14)

tower15 = Conv2D(filters = num_filter, kernel_size=(3,3),padding='same',kernel_initializer='glorot_uniform', activation='relu')(x4)
x = keras.layers.concatenate([tower12,tower13,tower14,tower15],axis=1)
x = AveragePooling2D(pool_size=(5,5),padding='valid', strides = (3,3))(x4)
x = Conv2D(64,(1,1))(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2,2))(x)
x = Flatten()(x)
x = Dense(1024)(x)
x = Dense(512)(x)
x = Dense(num_classes)(x)
output = Activation('softmax')(x)


In [10]:
model = Model([inputs],output)

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 61, 61, 8)    1184        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 31, 31, 8)    0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 31, 31, 8)    32          max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_2 (

In [13]:
opt = keras.optimizers.rmsprop(lr=0.0001,decay=1e-1)
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
model.fit(X_train,y1,batch_size=30,epochs=100,validation_split=0.1,shuffle=True)
score = model.evaluate(X_test, y2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('model.h5')
print('Saved model to disk')


Train on 3600 samples, validate on 400 samples
Epoch 1/100
3600/3600 [==============================] - 18s 5ms/step - loss: 0.9661 - acc: 0.5858 - val_loss: 1.0175 - val_acc: 0.4900
Epoch 2/100
3600/3600 [==============================] - 17s 5ms/step - loss: 0.8353 - acc: 0.6522 - val_loss: 1.0301 - val_acc: 0.4450
Epoch 3/100
3600/3600 [==============================] - 17s 5ms/step - loss: 0.8178 - acc: 0.6717 - val_loss: 0.9343 - val_acc: 0.6525
Epoch 4/100
3600/3600 [==============================] - 17s 5ms/step - loss: 0.8051 - acc: 0.6783 - val_loss: 0.9627 - val_acc: 0.5700
Epoch 5/100
3600/3600 [==============================] - 17s 5ms/step - loss: 0.7978 - acc: 0.6869 - val_loss: 0.9289 - val_acc: 0.6500
Epoch 6/100
3600/3600 [==============================] - 17s 5ms/step - loss: 0.7915 - acc: 0.6908 - val_loss: 0.9459 - val_acc: 0.5850
Epoch 7/100
3600/3600 [==============================] - 17s 5ms/step - loss: 0.7875 - acc: 0.6933 - val_loss: 0.9392 - val_acc: 0.5900
E